Tutorial link : https://pyimagesearch.com/2020/08/10/opencv-sudoku-solver-and-ocr/

# image preprocessing

In [1]:
!pip install opencv-python


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install tensorflow==2.15


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [70]:
!pip install imutils

  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25836 sha256=afc81f7d1e3042083385e90dd9db617e03e0eb22497a0685099592d90b5f7329
  Stored in directory: /home/users/harshit.sahu/.cache/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [92]:
import numpy as np
import cv2
import tensorflow as tf
import imutils

In [93]:
#read image as grayscale

image_path = "test1.png"
gray_image = cv2.imread("test1.png", cv2.IMREAD_GRAYSCALE)

In [95]:
#blur it slightly
image = gray_image
blurred = cv2.GaussianBlur(image, (7,7),3)

# apply adaptive thresholding and then invert the threshold map
thresh = cv2.adaptiveThreshold(
    blurred, 
    255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
    cv2.THRESH_BINARY, 
    11, 
    2
)
thresh = cv2.bitwise_not(thresh)

cv2.imwrite("xyz.png", thresh)

True

# find contours

In [96]:
# find contours in the thresholded image and sort them by size in
	# descending order
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

In [97]:
# initialize a contour that corresponds to the puzzle outline
puzzleCnt = None
# loop over the contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    # if our approximated contour has four points, then we can
    # assume we have found the outline of the puzzle
    if len(approx) == 4:
        puzzleCnt = approx
        break

In [98]:
# draw the contour of the puzzle on the image and then display
# it to our screen for visualization/debugging purposes
output = image.copy()
cv2.drawContours(output, [puzzleCnt], -1, (0, 255, 0), 2)
cv2.imwrite("Puzzle_Outline.png", output)

True

# warp image

In [100]:
# apply a four point perspective transform to both the original
# image and grayscale image to obtain a top-down bird's eye view
# of the puzzle

import imutils.perspective

# puzzle = imutils.perspective.four_point_transform(image, puzzleCnt.reshape(4, 2))
warped = imutils.perspective.four_point_transform(image, puzzleCnt.reshape(4, 2))
cv2.imwrite("warped_img.png", warped)

True

# extracting digits

In [101]:
warped.shape

(467, 591)

In [102]:
#make it square
#resizing it to 252*252, since we ultimately want 28*28 cells for detection
warped = cv2.resize(warped, (252, 252))
cv2.imwrite("warped_img.png", warped)
warped.shape

(252, 252)

In [103]:
cells = []

h, w = warped.shape
cell_h, cell_w = h//9, w//9

for i in range(9):
    row = []
    for j in range(9):
        cell = warped[i*cell_h:(i+1)*cell_h, j*cell_w:(j+1)*cell_w]        
        row.append(cell)
    cells.append(row)

cells

[[array([[126,  77,  81,  81,  77,  79,  80,  82,  82,  83,  82,  85,  90,
           93,  96,  98,  97,  97,  97,  99, 101, 103, 106, 107, 108, 107,
          106, 107],
         [116,  63, 102, 111, 109, 106, 104, 102, 100, 100, 100,  99,  98,
           97,  95,  92,  90,  87,  83,  82,  80,  80,  82,  82,  82,  83,
           83,  80],
         [117,  60, 128, 156, 157, 155, 154, 156, 154, 153, 154, 153, 151,
          149, 147, 146, 148, 148, 146, 146, 145, 144, 142, 140, 138, 139,
          141, 140],
         [116,  62, 130, 163, 165, 164, 163, 163, 162, 161, 161, 163, 164,
          164, 163, 163, 165, 165, 164, 164, 165, 166, 166, 166, 166, 165,
          164, 162],
         [121,  67, 129, 166, 166, 165, 164, 163, 162, 161, 161, 162, 161,
          166, 166, 167, 165, 163, 163, 164, 165, 166, 166, 166, 166, 166,
          165, 164],
         [122,  70, 126, 166, 165, 165, 162, 162, 161, 161, 162, 161, 159,
          160, 157, 158, 160, 163, 164, 165, 164, 165, 166, 166, 166, 

# digit detection

In [104]:
model = tf.keras.models.load_model("mnist_digits_cnn.keras")

OSError: No file or directory found at mnist_digits_cnn.keras

In [105]:
print(cells[0][0].shape)

(28, 28)


In [106]:
!pip install scikit-image
from skimage.segmentation import clear_border


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [107]:
sudoku_grid = []
i = 0
for row in cells:
    sudoku_row = []
    for cell in row:
        # apply automatic thresholding to the cell and then clear any
        # connected borders that touch the border of the cell
        thresh = cv2.threshold(cell, 0, 255,
            cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        thresh = clear_border(thresh)

        # find contours in the thresholded cell
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)


        # if no contours were found than this is an empty cell
        if len(cnts) == 0:
            sudoku_row.append(" ") 
        
        else:
            # otherwise, find the largest contour in the cell and create a
            # mask for the contour
            c = max(cnts, key=cv2.contourArea)
            mask = np.zeros(thresh.shape, dtype="uint8")
            cv2.drawContours(mask, [c], -1, 255, -1)

            # compute the percentage of masked pixels relative to the total
            # area of the image
            (h, w) = thresh.shape
            percentFilled = cv2.countNonZero(mask) / float(w * h)

            # if less than 3% of the mask is filled then we are looking at
            # noise and can safely ignore the contour
            if percentFilled < 0.03:
                sudoku_row.append(" ")

            else:
                # apply the mask to the thresholded cell
                digit = cv2.bitwise_and(thresh, thresh, mask=mask)
                cv2.imwrite(f"digits/img{i}.png", digit)
                i += 1
                
                digit = tf.keras.utils.img_to_array(digit)
                digit = np.expand_dims(digit, axis = 0)
                digit = np.expand_dims(digit, axis = -1)
                # digit = np.argmax(model.predict(digit))

                sudoku_row.append(digit)

    sudoku_grid.append(sudoku_row)


In [91]:
for row in sudoku_grid:
    print(row)

[8, ' ', ' ', ' ', 3, ' ', ' ', ' ', 9]
[' ', 5, ' ', 8, ' ', 8, ' ', 8, ' ']
[' ', ' ', 4, ' ', 9, ' ', 8, ' ', ' ']
[' ', 8, ' ', 3, ' ', 8, ' ', 2, ' ']
[5, ' ', 8, ' ', 8, ' ', 8, ' ', 2]
[' ', 9, ' ', 5, ' ', 2, ' ', 9, ' ']
[' ', ' ', 3, ' ', 4, ' ', 8, ' ', ' ']
[' ', 8, ' ', 3, ' ', 9, ' ', 4, ' ']
[3, ' ', ' ', ' ', 5, ' ', ' ', ' ', 8]
